## Exploring the school index info to generate historical and official win-loss percentage data.

### Data Sources

1. [Sports Reference CFB School Index](https://www.sports-reference.com/cfb/schools/)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

Inport the data.

In [2]:
data = pd.read_csv('Data/school_index_sr.csv')

Some general info about the dataset.

In [3]:
print(data.shape)
print(data.isnull().sum())
data.head()

(298, 20)
Rk          0
School      0
From        0
To          0
Yrs         0
G           0
W           0
L           0
T           0
Pct         0
G.1         0
W.1         0
L.1         0
T.1         0
Pct.1     135
SRS         0
SOS         0
AP          0
CC          0
Notes     231
dtype: int64


,Rk,School,From,To,Yrs,G,W,L,T,Pct,G.1,W.1,L.1,T.1,Pct.1,SRS,SOS,AP,CC,Notes
0,1,Air Force,1957,2019,63,705,384,325,12,0.541,26,12,13,1,0.481,1.02,-0.57,6,3,NaN
1,2,Akron,1975,2019,34,383,150,230,3,0.396,3,1,2,0,0.500,-11.46,-7.08,0,1,NaN
2,3,Alabama,1902,2019,116,1213,919,303,42,0.744,70,41,26,3,0.596,13.44,4.15,56,26,record adjusted to 890-312-41 by NCAA
3,4,Alameda Coast Guard,1943,1944,2,15,8,4,3,0.633,0,0,0,0,NaN,-0.16,-5.13,0,0,NaN
4,5,Alcorn State,1977,1977,1,11,3,8,0,0.273,0,0,0,0,NaN,-17.10,-9.19,0,0,NaN


Two things stand out to us immediately.  First, some of these schools are only represented from past seasons, but are not surrently competing.  See the feature column `'To'`.  We will filter our data to keep down to only teams competing in the updcoming season.  The other thing to note is that the information in the Win-Loss-Tie columns, along with the win percentage are not all officially accurate.  The NCAA sometimes, after reviewing a teams eligibality, decides that a teams wins must be [forfieted or vacated](https://www.sports-reference.com/cfb/friv/forfeits.html).  So, officially, there is an adjusted win percentage, while the one displayed in the column labeled `'Pct'` is more of an on the day of win percentage.

In what follows we will first remove the teams that are not currently competing.  Then we will create several new features for displaying the official win-loss-tie and win percentage data, taking into account the information in the `'Notes'` feature.

In [4]:
# defining the stuff_to_keep dataset that we will transform below.
stuff_to_keep = data[data['To'] == 2019]

In [5]:
print(stuff_to_keep.shape)
print(stuff_to_keep.isnull().sum())
stuff_to_keep.head()

(130, 20)
Rk         0
School     0
From       0
To         0
Yrs        0
G          0
W          0
L          0
T          0
Pct        0
G.1        0
W.1        0
L.1        0
T.1        0
Pct.1      5
SRS        0
SOS        0
AP         0
CC         0
Notes     67
dtype: int64


,Rk,School,From,To,Yrs,G,W,L,T,Pct,G.1,W.1,L.1,T.1,Pct.1,SRS,SOS,AP,CC,Notes
0,1,Air Force,1957,2019,63,705,384,325,12,0.541,26,12,13,1,0.481,1.02,-0.57,6,3,NaN
1,2,Akron,1975,2019,34,383,150,230,3,0.396,3,1,2,0,0.500,-11.46,-7.08,0,1,NaN
2,3,Alabama,1902,2019,116,1213,919,303,42,0.744,70,41,26,3,0.596,13.44,4.15,56,26,record adjusted to 890-312-41 by NCAA
7,8,Appalachian State,1972,2019,16,174,97,72,5,0.572,4,4,0,0,1.000,-3.06,-6.96,0,1,NaN
8,9,Arizona,1931,2019,86,920,486,422,23,0.534,20,9,10,1,0.500,1.72,0.59,8,6,NaN


Notice that there are 67 nulls in the `'Notes'` feature.  Let's explore the different types of information that are housed in this column.

In [6]:
# first, extend the number of displayed rows
pd.options.display.max_rows = 200

In [7]:
# now, reset the index
stuff_to_keep.reset_index(inplace=True)
stuff_to_keep.head()

,index,Rk,School,From,To,Yrs,G,W,L,T,...,G.1,W.1,L.1,T.1,Pct.1,SRS,SOS,AP,CC,Notes
0,0,1,Air Force,1957,2019,63,705,384,325,12,...,26,12,13,1,0.481,1.02,-0.57,6,3,NaN
1,1,2,Akron,1975,2019,34,383,150,230,3,...,3,1,2,0,0.500,-11.46,-7.08,0,1,NaN
2,2,3,Alabama,1902,2019,116,1213,919,303,42,...,70,41,26,3,0.596,13.44,4.15,56,26,record adjusted to 890-312-41 by NCAA
3,7,8,Appalachian State,1972,2019,16,174,97,72,5,...,4,4,0,0,1.000,-3.06,-6.96,0,1,NaN
4,8,9,Arizona,1931,2019,86,920,486,422,23,...,20,9,10,1,0.500,1.72,0.59,8,6,NaN


Now, drop the unneeded `'index'` column.

In [8]:
stuff_to_keep.drop(columns='index', axis=1, inplace=True)
stuff_to_keep.head()

/Users/blakewallace/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Rk,School,From,To,Yrs,G,W,L,T,Pct,G.1,W.1,L.1,T.1,Pct.1,SRS,SOS,AP,CC,Notes
0,1,Air Force,1957,2019,63,705,384,325,12,0.541,26,12,13,1,0.481,1.02,-0.57,6,3,NaN
1,2,Akron,1975,2019,34,383,150,230,3,0.396,3,1,2,0,0.500,-11.46,-7.08,0,1,NaN
2,3,Alabama,1902,2019,116,1213,919,303,42,0.744,70,41,26,3,0.596,13.44,4.15,56,26,record adjusted to 890-312-41 by NCAA
3,8,Appalachian State,1972,2019,16,174,97,72,5,0.572,4,4,0,0,1.000,-3.06,-6.96,0,1,NaN
4,9,Arizona,1931,2019,86,920,486,422,23,0.534,20,9,10,1,0.500,1.72,0.59,8,6,NaN


Next, we explore the data in the `'Notes'` feature.

In [9]:
stuff_to_keep[stuff_to_keep['Notes'].notnull()]

,Rk,School,From,To,Yrs,G,W,L,T,Pct,G.1,W.1,L.1,T.1,Pct.1,SRS,SOS,AP,CC,Notes
2,3,Alabama,1902,2019,116,1213,919,303,42,0.744,70,41,26,3,0.596,13.44,4.15,56,26,record adjusted to 890-312-41 by NCAA
6,11,Arkansas,1906,2019,114,1169,681,481,37,0.583,42,15,24,3,0.393,6.58,2.99,28,14,record adjusted to 682-480-37 by NCAA
7,12,Arkansas State,1974,2019,38,431,193,236,2,0.450,9,3,6,0,0.375,-9.25,-6.99,0,6,record adjusted to 183-236-2 by NCAA
9,14,Auburn,1902,2019,117,1190,741,431,43,0.628,43,24,17,2,0.595,8.78,4.41,39,8,record adjusted to 744-429-42 by NCAA
11,17,Baylor,1903,2019,114,1176,584,565,41,0.508,24,13,11,0,0.542,2.63,2.61,14,8,record adjusted to 584-565-40 by NCAA
17,33,California,1916,2019,104,1105,571,513,31,0.526,23,11,11,1,0.500,5.53,4.17,14,13,record adjusted to 568-512-31 by NCAA
24,55,Colorado State,1902,2019,112,1095,515,556,31,0.481,17,6,11,0,0.375,-3.80,-2.97,3,15,record adjusted to 516-555-31 by NCAA
27,73,East Carolina,1965,2019,55,611,320,296,3,0.519,16,7,9,0,0.438,-2.61,-2.81,2,6,record adjusted to 321-295-3 by NCAA
32,81,Florida State,1954,2019,66,743,519,236,16,0.684,46,28,16,2,0.622,11.32,4.03,33,15,record adjusted to 508-235-16 by NCAA
33,87,Fresno State,1969,2019,51,602,350,251,4,0.582,23,11,12,0,0.455,-1.77,-5.29,3,11,record adjusted to 351-250-4 by NCAA


Here is a look at how the win percentage is calculated.  Note that each tie is counted as a half win.

In [42]:
(2*919+42)/(2*(919+303+42))

0.7436708860759493

In [43]:
(2*890+41)/(2*(890+312+41))

0.7325020112630732

In [10]:
stuff_to_keep['Notes'][2:3]

2    record adjusted to 890-312-41 by NCAA
Name: Notes, dtype: object

Now, we create a dictionary to hold the index and the official record.

In [11]:
dict_index_record = {}

for index, value in stuff_to_keep['Notes'].items():
    if type(value) == str:
#         print(index, value[19:29])
        dict_index_record[index] = value[19:29].split('-')
dict_index_record
#         win_loss_tie = value[19:29].split('-')
#         dict_win_loss_tie[index] = 
#         print(win_loss_tie)

{2: ['890', '312', '41'],
 6: ['682', '480', '37'],
 7: ['183', '236', '2 '],
 9: ['744', '429', '42'],
 11: ['584', '565', '40'],
 17: ['568', '512', '31'],
 24: ['516', '555', '31'],
 27: ['321', '295', '3 '],
 32: ['508', '235', '16'],
 33: ['351', '250', '4 '],
 37: ['728', '481', '40'],
 39: ['429', '346', '14'],
 41: ['460', '655', '38'],
 42: ['620', '529', '34'],
 43: ['512', '641', '45'],
 45: ['472', '600', '34'],
 47: ['492', '551', '36'],
 49: ['205', '295', '5 '],
 50: ['230', '210', '7 '],
 52: ['351', '296', '7 '],
 53: ['769', '400', '47'],
 57: ['315', '337', '8 '],
 59: ['351', '277', '15'],
 61: ['573', '382', '30'],
 64: ['555', '566', '36'],
 65: ['631', '529', '50'],
 67: ['863', '363', '37'],
 69: ['161', '314', '3 '],
 71: ['315', '578', '14'],
 72: ['631', '514', '48'],
 74: ['261', '327', '11'],
 76: ['536', '658', '40'],
 77: ['820', '310', '33'],
 79: ['791', '257', '36'],
 83: ['612', '501', '34'],
 84: ['580', '454', '34'],
 85: ['462', '562', '36'],
 87: 

Next, convert the values in each list to integer values (at the moment they are strings, which is indicated by the quotes around them).

In [12]:
for item in dict_index_record:
    for i in range(len(dict_index_record[item])):
        dict_index_record[item][i] = int(dict_index_record[item][i])
#     print(dict_index_record[item])
dict_index_record

{2: [890, 312, 41],
 6: [682, 480, 37],
 7: [183, 236, 2],
 9: [744, 429, 42],
 11: [584, 565, 40],
 17: [568, 512, 31],
 24: [516, 555, 31],
 27: [321, 295, 3],
 32: [508, 235, 16],
 33: [351, 250, 4],
 37: [728, 481, 40],
 39: [429, 346, 14],
 41: [460, 655, 38],
 42: [620, 529, 34],
 43: [512, 641, 45],
 45: [472, 600, 34],
 47: [492, 551, 36],
 49: [205, 295, 5],
 50: [230, 210, 7],
 52: [351, 296, 7],
 53: [769, 400, 47],
 57: [315, 337, 8],
 59: [351, 277, 15],
 61: [573, 382, 30],
 64: [555, 566, 36],
 65: [631, 529, 50],
 67: [863, 363, 37],
 69: [161, 314, 3],
 71: [315, 578, 14],
 72: [631, 514, 48],
 74: [261, 327, 11],
 76: [536, 658, 40],
 77: [820, 310, 33],
 79: [791, 257, 36],
 83: [612, 501, 34],
 84: [580, 454, 34],
 85: [462, 562, 36],
 87: [676, 481, 37],
 88: [604, 562, 48],
 89: [465, 615, 32],
 90: [479, 477, 22],
 91: [324, 260, 8],
 92: [334, 417, 14],
 93: [487, 536, 54],
 95: [576, 545, 41],
 97: [360, 278, 6],
 99: [663, 490, 41],
 100: [317, 411, 26],
 101:

Now, we take each record and create a dictionary that house the win-loss-tie data.

In [14]:
# sample code
lst1 = [4, 7, 2]
lst2 = [3,6, 5]

outcome = ['win', 'loss', 'tie']
inner_dict = {}
dict_prac = {}
lst = [lst1, lst2]
for thing in range(len(lst)): 
    for j in range(len(lst[thing])):
#         print(j)
        
        inner_dict[outcome[j]] = lst[thing][j]
    dict_prac[str(lst[thing])] = inner_dict
    inner_dict = {}
dict_prac

{'[4, 7, 2]': {'win': 4, 'loss': 7, 'tie': 2},
 '[3, 6, 5]': {'win': 3, 'loss': 6, 'tie': 5}}

The next code block generates a dictionary housing the official win-loss-tie data in a dictionary, `dict_win_loss_tie`.

In [18]:
outcome = ['win', 'loss', 'tie']
inner_dict = {}
dict_win_loss_tie = {}

for thing in dict_index_record: 
    for j in range(len(dict_index_record[thing])):
#         print(j)
        
        inner_dict[outcome[j]] = dict_index_record[thing][j]
    dict_win_loss_tie[thing] = inner_dict
    inner_dict = {}
dict_win_loss_tie

{2: {'win': 890, 'loss': 312, 'tie': 41},
 6: {'win': 682, 'loss': 480, 'tie': 37},
 7: {'win': 183, 'loss': 236, 'tie': 2},
 9: {'win': 744, 'loss': 429, 'tie': 42},
 11: {'win': 584, 'loss': 565, 'tie': 40},
 17: {'win': 568, 'loss': 512, 'tie': 31},
 24: {'win': 516, 'loss': 555, 'tie': 31},
 27: {'win': 321, 'loss': 295, 'tie': 3},
 32: {'win': 508, 'loss': 235, 'tie': 16},
 33: {'win': 351, 'loss': 250, 'tie': 4},
 37: {'win': 728, 'loss': 481, 'tie': 40},
 39: {'win': 429, 'loss': 346, 'tie': 14},
 41: {'win': 460, 'loss': 655, 'tie': 38},
 42: {'win': 620, 'loss': 529, 'tie': 34},
 43: {'win': 512, 'loss': 641, 'tie': 45},
 45: {'win': 472, 'loss': 600, 'tie': 34},
 47: {'win': 492, 'loss': 551, 'tie': 36},
 49: {'win': 205, 'loss': 295, 'tie': 5},
 50: {'win': 230, 'loss': 210, 'tie': 7},
 52: {'win': 351, 'loss': 296, 'tie': 7},
 53: {'win': 769, 'loss': 400, 'tie': 47},
 57: {'win': 315, 'loss': 337, 'tie': 8},
 59: {'win': 351, 'loss': 277, 'tie': 15},
 61: {'win': 573, 'los

From the above variable, `dict_win_loss_tie`, we create feature variables that capture the official `'adjusted'` outcomes.